![New Relic](./assets/new-relic-logo.png)

# New Relic Playground

## Imports

Python Imports

In [ ]:
import json
import os

from textwrap import dedent

Third-party libraries

In [ ]:
import dotenv
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import requests

from requests import Session, request

## Useful definitions

In [ ]:
NULL_CURSOR = json.dumps(None)

In [ ]:
def build_query(query_string, query_params = None):
    if query_params is None:
        query_params = {}
        
    return dedent(query_string.strip()) % query_params

In [ ]:
def get_response_data(response, key_path = None, action = "actor"):
    data = response.json().get("data").get(action)

    if key_path is not None:
        for key in key_path.split(":"):
            if key.isdecimal() and isinstance(data, list):
                data = data[int(key)]
            else:
                data = data.get(key)

    return data

In [ ]:
class NewRelicGqlClient(Session):
    """Client for New Relic GraphQL API."""

    url: str = "https://api.newrelic.com/graphql"

    def __init__(self, new_relic_user_key = None):
        super().__init__()

        self.headers.update(
            {
                "Content-Type": "application/json",
                "API-Key": new_relic_user_key,
            }
        )

    def execute(self, query, variables = None, **kwargs):
        data = json.dumps(
            {
                "query": query,
                "variables": variables,
            },
        )
        return self.post(self.url, data=data, **kwargs)

## Client setup

Load credentials

In [ ]:
env_file = "../.env"

dotenv.load_dotenv(env_file)

new_relic_user_key = os.environ.get("NEW_RELIC_USER_KEY", None)

if new_relic_user_key is None:
    raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

Client instantiation

In [ ]:
newrelic = NewRelicGqlClient(new_relic_user_key)

Define test query

In [ ]:
query = build_query(
    """
        {
            actor {
                user {
                    email
                    name
                    id
                }
            }
        }
    """
)

Executing query

In [ ]:
response = newrelic.execute(query)

Looking for response data

In [ ]:
print(
    json.dumps(
        get_response_data(
            response, 
            action="actor",
        ),
        indent=4,
    )
)

## Playground area

In [ ]:
ACCOUNT_ID = put your account id here

### List all available policies

In [ ]:
query_string="""
{
    actor {
        account(id: %(account_id)d) {
            alerts {
                policiesSearch(cursor: %(cursor)s) {
                    policies {
                        id
                        incidentPreference
                        name
                    }
                    totalCount
                    nextCursor
                }
            }
        }
    }
}
"""

query_params={
    "account_id": ACCOUNT_ID,
    "cursor": NULL_CURSOR,
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
policies_count = get_response_data(response, key_path="account:alerts:policiesSearch:totalCount", action="actor")
policies_count

In [ ]:
policies_list = get_response_data(response, key_path="account:alerts:policiesSearch:policies", action="actor")
len(policies_list)

In [ ]:
policies_list[4]

In [ ]:
next_cursor = get_response_data(response, key_path="account:alerts:policiesSearch:nextCursor", action="actor")
next_cursor

### Get a single policy

In [ ]:
policy_id = policies_list[4]["id"]

In [ ]:
query_string="""
{
    actor {
        account(id: %(account_id)d) {
            alerts {
                policy(id: %(policy_id)s) {
                    id
                    incidentPreference
                    name
                }
            }
        }
    }
}
"""

query_params={
    "account_id": ACCOUNT_ID,
    "policy_id": policy_id,
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
policy = get_response_data(response, key_path="account:alerts:policy", action="actor")
policy

### Get all NRQL conditions

In [ ]:
policy_id = policies_list[4]["id"]

In [ ]:
query_string="""
{
    actor {
        account(id: %(account_id)d) {
            alerts {
                nrqlConditionsSearch {
                    nextCursor
                    totalCount
                    nrqlConditions {
                        id
                        name
                        nrql {
                            query
                        }
                        policyId
                    }
                }
            }
        }
    }
}
"""

query_params={
    "account_id": ACCOUNT_ID,
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
conditions_count = get_response_data(response, key_path="account:alerts:nrqlConditionsSearch:totalCount", action="actor")
conditions_count

In [ ]:
conditions_list = get_response_data(response, key_path="account:alerts:nrqlConditionsSearch:nrqlConditions", action="actor")
len(conditions_list)

In [ ]:
conditions_list[0]

In [ ]:
next_cursor = get_response_data(response, key_path="account:alerts:nrqlConditionsSearch:nextCursor", action="actor")
next_cursor

### Get conditions from a policy

In [ ]:
condition_id = conditions_list[0]["id"]

In [ ]:
query_string="""
{
    actor {
        account(id: %(account_id)d) {
            alerts {
                nrqlCondition(id: %(condition_id)s) {
                    description
                    enabled
                    id
                    name
                    nrql {
                        query
                    }
                    policyId
                    runbookUrl
                    type
                    terms {
                        operator
                        priority
                        threshold
                        thresholdDuration
                        thresholdOccurrences
                    }
                }
            }
        }
    }
}

"""

query_params={
    "account_id": ACCOUNT_ID,
    "condition_id": condition_id,
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
condition = get_response_data(response, key_path="account:alerts:nrqlCondition", action="actor")
condition

In [ ]:
condition["policyId"]

In [ ]:
condition["nrql"]["query"]

### Create a policy

In [ ]:
query_string="""
mutation {
    alertsPolicyCreate(
        accountId: %(account_id)d, 
        policy: {
            incidentPreference: PER_POLICY, 
            name: "Sample API Policy"
        }
    ) {
        accountId
        incidentPreference
        name
        id
    }
}
"""

query_params={
    "account_id": ACCOUNT_ID,
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
created_policy = get_response_data(response, action="alertsPolicyCreate")
created_policy

### Append nrql condition to policy

In [ ]:
policy_id = created_policy["id"]

In [ ]:
query_string="""
mutation {
    alertsNrqlConditionStaticCreate(
        accountId: %(account_id)d, 
        policyId: %(policy_id)s, 
        condition: {
            enabled: false, 
            name: "Sample NRQL condition from API", 
            nrql: {
                query: "SELECT average(duration) FROM Transaction"
            }, 
            terms: [
                {
                    operator: ABOVE, 
                    priority: CRITICAL, 
                    threshold: 10, 
                    thresholdDuration: 300, 
                    thresholdOccurrences: ALL
                }
            ]
        }
    ) {
        name
        policyId
        nrql {
            query
        }
        terms {
            operator
            threshold
            thresholdDuration
            thresholdOccurrences
            priority
        }
        id
    }
}
"""

query_params={
    "account_id": ACCOUNT_ID,
    "policy_id": policy_id,
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
created_policy = get_response_data(response, action="alertsNrqlConditionStaticCreate")
created_policy